### Extract device settings as CSV files for `Settings Manager` from an XLSX file prepared for the beam envelope controling during the cavity phase scan tuning.

In [1]:
import xlrd
from phantasy import MachinePortal
from phantasy import disable_warnings
from phantasy_apps.settings_manager.contrib import SettingsRow

disable_warnings()

In [2]:
# input parameters for LS1 cavity tuning
machine = "FRIB"
segments = ["MEBT_FS1A"]

xlsx_filepath = "20200224_phase_scan_setting_list_maruta.xlsx"
sheet_name = "36Ar10 magnets"
row_start = 4
column_start = 4
settings_rootpath = "36Ar10_LS1"
snp_tags = "LS1-phasing"
snp_machine = "FRIB"
snp_segment = "LINAC"
snp_ion_name = "Ar"
snp_ion_mass = 36
snp_ion_charge = 10
snp_ion_number = 18
snp_user = "zhangt"
snp_app_name = "Settings Manager"
snp_app_version = "4.1"

In [15]:
# input parameters for LS2 cavity tuning
machine = "FRIB"
segments = ["MEBT_FS1B", "STRIPPER_FS2A"]

xlsx_filepath = "ARR04_phase_scan_ver20200322.xlsx"
sheet_name = "129Xe LS2 phase scan magnet"
row_start = 4
column_start = 4
settings_rootpath = "129Xe49_LS2"

snp_tags = "LS2-phasing"
snp_machine = "FRIB"
snp_segment = "LINAC"
snp_ion_name = "Xe"
snp_ion_mass = 129
snp_ion_charge = 49
snp_ion_number = 54
snp_user = "zhangt"
snp_app_name = "Settings Manager"
snp_app_version = "4.1"

In [2]:
# input parametes for full acceleration cases of 129Xe 27+, MEBT to FS1A
machine = "FRIB"
segments = ["LINAC"]

xlsx_filepath = "ARR04_2020Oct_ver20201005.xlsx"
sheet_name = "129Xe27_FS1A"
row_start = 4
column_start = 4
settings_rootpath = "129Xe27_50"

snp_tags = "LS1-phasing"
snp_note = "Full acceleration 129Xe27+ to 17.0 MeV from MEBT to FS1A"
snp_machine = "FRIB"
snp_segment = "LINAC"
snp_ion_name = "Xe"
snp_ion_mass = 129
snp_ion_charge = 27
snp_ion_number = 54
snp_user = "zhangt"
snp_app_name = "Settings Manager"
snp_app_version = "4.1"

In [ ]:
# input parametes for full acceleration cases of 129Xe 50+ MEBT to FS1B
machine = "FRIB"
segments = ["LINAC"]

xlsx_filepath = "ARR04_2020Oct_ver20201005.xlsx"
sheet_name = "129Xe27-50_FS1B"
row_start = 4
column_start = 4
settings_rootpath = "129Xe27_50"

snp_tags = "LS1-phasing"
snp_note = "Full acceleration 129Xe50+ to 16.5 MeV from MEBT to FS1B"
snp_machine = "FRIB"
snp_segment = "LINAC"
snp_ion_name = "Xe"
snp_ion_mass = 129
snp_ion_charge = 50
snp_ion_number = 54
snp_user = "zhangt"
snp_app_name = "Settings Manager"
snp_app_version = "4.1"

In [ ]:
# input parametes for full acceleration cases of 129Xe 50+ MEBT to FS2
machine = "FRIB"
segments = ["LINAC"]

xlsx_filepath = "ARR04_2020Oct_ver20201005.xlsx"
sheet_name = "129Xe27-50_FS1B"
row_start = 4
column_start = 4
settings_rootpath = "129Xe27_50_FS2"

snp_tags = "LS2-phasing"
snp_note = "Full acceleration 129Xe50+ to 180 MeV from MEBT to FS2"
snp_machine = "FRIB"
snp_segment = "LINAC"
snp_ion_name = "Xe"
snp_ion_mass = 129
snp_ion_charge = 50
snp_ion_number = 54
snp_user = "zhangt"
snp_app_name = "Settings Manager"
snp_app_version = "4.1"

In [3]:
wkbk = xlrd.open_workbook(xlsx_filepath)
if sheet_name not in wkbk.sheet_names():
    raise RuntimeError("Cannot find sheet {} from given xlsx file.".format(sheet_name))
magnets_settings = wkbk.sheet_by_name(sheet_name)
head_row = magnets_settings.row(1)
head_row;

In [4]:
mp = MachinePortal(machine, segments[0]);
for segm in segments[1:]:
    mp.load_lattice(segm)
lat = mp.combined_lattice()
lat;

In [5]:
# get a list of column index of element name and element object.
# [(cell index, CaElement)]
index_elem_list = []
col_idx0 = column_start - 1
for i, cell in enumerate(head_row[col_idx0:], col_idx0):
    if cell.value == 'STOP':
        break
    elem = lat.get_elements(name="*{}*".format(cell.value))
    if elem:
        index_elem_list.append((i, elem[0]))

In [6]:
list_settings_row = []
last_settings_row = None
for ridx in range(row_start - 1, magnets_settings.nrows):
    row = magnets_settings.row(ridx)
    current_settings_row = SettingsRow(row, index_elem_list, last_settings_row,
                                       ion_name=snp_ion_name, ion_number=snp_ion_number,
                                       ion_mass=snp_ion_mass, ion_charge=snp_ion_charge,
                                       machine=snp_machine, segment=snp_segment, tags=snp_tags,
                                       app=snp_app_name, version=snp_app_version, user=snp_user)
    if not current_settings_row == last_settings_row:
        current_settings_row.write(rootpath=settings_rootpath, row_id=ridx + 1)
    last_settings_row = current_settings_row

KeyboardInterrupt: 